In [1]:
import pdfplumber
import os
import re
import pandas as pd
import numpy as np

# pd.options.display.max_rows = 9999

In [4]:
pl_text = [  '四半期連結損益及び包括利益計算書'
           , '四半期連結損益計算書'
           , '四半期連結損益及び包括利益計算書'
           , '連結損益及び包括利益計算書'
           , '連結損益計算書']

bs_text = [  '四半期連結貸借対照表'
           , '連結貸借対照表'
           , '連結財政状態計算書'
           , '連結貸借対照表']

def find_header(pdf,
                page_id: int, n_cols):
    page = pdf.pages[page_id]
    text = page.extract_text()
    DON_VI = '単位'

    idx_dv = text.find(DON_VI)
    if idx_dv >= 0:
        text1 = text[idx_dv:]
        ## Tim don vi
        donviYen = text1[3: text1.find('\n') - 1]
        ## Tim field
        text2 = text1[text1.find('\n') + 1: ]
        text3 = text2[: text2.find('\n')]
        text3 = text3.split(' ')

        if len(text3) != n_cols - 1:
            return [donviYen] + ['B'] *(n_cols - 1)
        return [donviYen] + text3
    else:
        return ['N/A']*n_cols

def check_str(text, lst_str):
    for s in lst_str:
        idx = text.find(s)
        if idx != -1:
            return idx < text.find('単位')
    return False

def convert_text_to_table(text, n_cols):
    rows = text.split('\n')
    lst_table = []
    for row in rows:
        row_ = row.split(' ')
        if len(row_) <= n_cols:
            if len(row_) == 1:
                row_ += [None] *(n_cols - 1)
            else:
                row_ = [None] * (n_cols - len(row_)) + row_
            lst_table += [row_]
    return lst_table

def extract_data_pdf(pdf,
                     lst_table = [],
                     page_id = 0,
                     type_data = 'find_keyword',
                     lst_str = []):

    page = pdf.pages[page_id]
    tables = page.extract_tables()
    text = page.extract_text()
    text = text.replace(" ","")

    if type_data == 'find_keyword':
        find_key = check_str(text, lst_str)
    if type_data == 'avoid_keyword':
        idx = text.find('\n')
        if text[idx + 2: idx + 4] == '単位':
            find_key = True
        else: find_key = False

    n_cols = 0
    if ((find_key == True)
        and (len(tables) > 0)):
            n_cols = len(tables[0][0])
            for table in tables:
                for row in table:
                    row_ = [[x for x in row[:-(n_cols - 1)] if (x != None and x != '')] + row[-(n_cols -1):]]
                    if len(row_[0]) != n_cols:
                        row_[0] = ['B'] * n_cols
                    lst_table += row_
            return lst_table, n_cols, True

    # if find_key == True and (len(tables) > 0):
    #     n_cols = len(tables[0][0])
    #     # print(n_cols)
    #     text = page.extract_text()
    #     t = text[text.find('単位'): ]
    #     lst_table = convert_text_to_table(t, n_cols)
    #     return lst_table, n_cols, True
    return lst_table, n_cols, False

def extract_pdf(path_file, key):
    pdf = pdfplumber.open(path_file)
    for page_id in range(len(pdf.pages)):
        lst_table, n_cols, _ = extract_data_pdf(pdf,
                                        lst_table = [],
                                        page_id = page_id,
                                        type_data = 'find_keyword',
                                        lst_str = key )
        if _ == True:
            lst_col = find_header(pdf, page_id = page_id, n_cols=n_cols)
            break
    
    while True:
        # print(page_id)
        lst_table, n_cols, _ = extract_data_pdf(pdf,
                                        lst_table = lst_table,
                                        page_id = page_id+1,
                                        type_data = 'avoid_keyword',
                                        lst_str = [] )
        if _ == True:
            page_id += 1
        else: break
    # print(lst_table)
    return pd.DataFrame(lst_table, columns = lst_col)

def get_table(id_company, type_ = 'bs',
               path_save = '', 
               return_df = False, 
               save_file = True):
    if type_ == 'bs': 
        key = bs_text
    elif type_ == 'pl':
        key = pl_text

    for file in os.listdir(path_save + f'Data/{id_company}/PDF'):
        if 'ocr' not in file and '(訂正)' not in file:
            try:
                numbers = re.findall(r'\d+', file)
                year = numbers[0]
                quy = f'Q{numbers[1]}'
                path_file = path_save + f'Data/{id_company}/PDF/{file}'
                df_ = extract_pdf(path_file, key)
                if save_file:
                    df_.to_csv(path_save + f'Data/{id_company}/docs/{type_}/{year}_{quy}.csv', index=False)
                if return_df:
                    print(f'Data/{id_company}/{type_}/{year}_{quy}')
                    print(df_)
                    print(' ')
            except:
                print(f'----------BUG---------{file}')


In [5]:
path_file = "tests/Data/1333/PDF/2018_Q2_決算短信(2018_11_5).pdf"
pdf = pdfplumber.open(path_file)
# page = pdf.pages[6]
# text = page.extract_text()
# t = text[text.find('単位'):]
# print(t)
# convert_text_to_table(t, 3)
# check_str(text, pl_text)

# extract_data_pdf(pdf,lst_table = [], page_id = 7,
#                                         type_data = 'find_keyword',
#                                         lst_str = pl_text )
# tables = page.extract_tables()
# for table in tables:
#         # Xử lý bảng theo nhu cầu của bạn
#         # Ví dụ: in nội dung bảng
#         for row in table:
#             print(row)
#         print("-------------")
extract_pdf(path_file, pl_text)

,百万円,前第２四半期連結累計期間,当第２四半期連結累計期間
0,B,B,B
1,B,B,B
2,売上高,"446,771","448,549"
3,売上総利益,"62,830","61,166"
4,営業利益,"12,963","10,215"
5,受取配当金,697,610
6,営業外収益合計,"3,226","2,909"
7,支払利息,911,862
8,営業外費用合計,"1,106","1,227"
9,特別利益,,


In [6]:
get_table(1301, type_ = 'pl', path_save = 'tests/', return_df = True, save_file = False)

Data/1301/pl/2008_Q1
                              百万円    前第１四半期連結累計期間
0                               B               B
1                               B               B
2                           Ⅰ 売上高          37,204
3                   Ⅱ 売上原価\n売上総利益   32,854\n4,349
4                    Ⅲ 販売費及び一般管理費           3,591
5                            営業利益             758
6       Ⅳ 営業外収益\n受取利息\n受取配当金\nその他  117\n8\n53\n55
7              Ⅴ 営業外費用\n支払利息\nその他     128\n127\n1
8                            経常利益             747
9                  Ⅵ 特別利益\nⅦ 特別損失           10\n1
10                   税金等調整前四半期純利益             757
11  法人税、住民税及び事業税\n法人税等調整額\n法人税等合計    257\n42\n299
12                         少数株主利益              18
13                         四半期純利益             439
 
Data/1301/pl/2008_Q2
                  百万円 当第２四半期連結累計期間
0                 売上高       73,644
1                売上原価       63,775
2               売上総利益        9,868
3          販売費及び一般管理費             
4            賞与引当金繰入額          

In [ ]:
df.loc[27]

単位：百万円     受取保険金
前連結会計年度        －
当連結会計年度       45
Name: 27, dtype: object

In [ ]:
find_header(pdf,7)

['単位：百万円', '前連結会計年度', '当連結会計年度']

In [ ]:
extract_data_pdf(pdf,
                     lst_table = [],
                     page_id = 7,
                     type_data = 'find_keyword',
                     key_word = bs_text)

([['資産の部', '', ''],
  ['流動資産', '', ''],
  ['現金及び預金', '6,544', '7,050'],
  ['受取手形及び売掛金', '28,683', '33,079'],
  ['商品及び製品', '44,997', '52,620'],
  ['仕掛品', '3,440', '3,840'],
  ['原材料及び貯蔵品', '6,191', '7,425'],
  ['その他', '9,678', '10,196'],
  ['貸倒引当金', '△9', '△9'],
  ['流動資産合計', '99,527', '114,202'],
  ['固定資産', '', ''],
  ['有形固定資産', '', ''],
  ['建物及び構築物（純額）', '6,228', '7,057'],
  ['機械装置及び運搬具（純額）', '3,844', '3,853'],
  ['船舶（純額）', '347', '3,069'],
  ['土地', '3,784', '3,910'],
  ['リース資産（純額）', '307', '304'],
  ['建設仮勘定', '3,944', '335'],
  ['その他（純額）', '440', '382'],
  ['有形固定資産合計', '18,897', '18,912'],
  ['無形固定資産', '', ''],
  ['リース資産', '1', '0'],
  ['その他', '348', '347'],
  ['無形固定資産合計', '350', '348'],
  ['投資その他の資産', '', ''],
  ['投資有価証券', '9,272', '10,134'],
  ['繰延税金資産', '1,136', '1,454'],
  ['その他', '2,873', '2,845'],
  ['貸倒引当金', '△1,597', '△1,597'],
  ['投資その他の資産合計', '11,685', '12,837'],
  ['固定資産合計', '30,932', '32,098'],
  ['資産合計', '130,460', '146,301']],
 True)

In [ ]:
page = pdf.pages[11]
tables = page.extract_table()
len(tables)
text = page.extract_text()
text.replace(" ","")

'株式会社極洋(1301)2023年３月期決算短信\n（３）連結株主資本等変動計算書\n前連結会計年度(自2021年４月１日至2022年３月31日)\n(単位：百万円)\n株主資本\n資本金資本剰余金利益剰余金自己株式株主資本合計\n当期首残高5,6641,32928,737△43035,300\n当期変動額\n剰余金の配当△864△864\n親会社株主に帰属する\n4,6344,634\n当期純利益\n自己株式の取得△0△0\n自己株式の処分11\n連結子会社株式の取得\n11\nによる持分の増減\n株主資本以外の項目の\n当期変動額（純額）\n当期変動額合計―13,77003,771\n当期末残高5,6641,33032,507△42939,072\nその他の包括利益累計額\nその他有価証券繰延ヘッジ為替換算退職給付に係るその他の非支配株主持分純資産合計\n包括利益\n評価差額金損益調整勘定調整累計額\n累計額合計\n当期首残高5,13914895△3015,081△40739,975\n当期変動額\n剰余金の配当△864\n親会社株主に帰属する\n4,634\n当期純利益\n自己株式の取得△0\n自己株式の処分1\n連結子会社株式の取得\n1\nによる持分の増減\n株主資本以外の項目の\n△1,90419419664△1,448△123△1,572\n当期変動額（純額）\n当期変動額合計△1,90419419664△1,448△1232,199\n当期末残高3,234343292△2373,632△53042,174\n-10-'